In [ ]:
import numpy as np

import tensorflow as tf
import time
import os
import glob
import PIL
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
import cv2
import pandas as pd
import tensorflow.keras
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import img_to_array,ImageDataGenerator
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, Flatten,Input
from tensorflow.keras.layers import Conv2D, MaxPooling2D,BatchNormalization,AveragePooling2D
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
!unzip -q '/content/drive/My Drive/Classification/GalaxyClassification.zip'

In [ ]:
from keras.models import Model, Sequential
from keras.layers import Flatten, Dense, Add, Lambda, Input, Reshape, Conv2D, MaxPooling2D, BatchNormalization, Activation, ZeroPadding2D, GlobalMaxPooling2D, Dropout, AveragePooling2D
import tensorflow as tf

In [ ]:
classes = ["class 0","class 1","class 2","class 3","class 4", "class 5","class 6"]

def append_ext(fn):
    return fn + ".jpg"


traindf = pd.read_csv("/content/datasets/train.csv")
traindf["id"] = traindf['GalaxyID'].astype(str).apply(append_ext)

In [ ]:
train_path = "/content/images/train_pool/"

In [ ]:
def random_input(img):
    shape = img.shape[:2]
    left = int(shape[0]/4)
    top = int(shape[1]/4)
    img = img[left:left*3,top:top*3,:]
    image = cv2.resize(img, shape, interpolation = cv2.INTER_CUBIC)
    image = img_to_array(image)
    return image


datagen = ImageDataGenerator(
    rescale=1. / 255,
    rotation_range=140,
    width_shift_range=0.1,
    height_shift_range=0.1,
    brightness_range = (0.9, 2),
    samplewise_std_normalization=False,
    horizontal_flip=True,
    vertical_flip=True,
    validation_split = 0.15,
    preprocessing_function = random_input,
)

valid_datagen=ImageDataGenerator(
    rescale=1./255,
    validation_split=0.15,
    preprocessing_function = random_input,
    )

In [ ]:
train_generator = datagen.flow_from_dataframe(
    dataframe=traindf,
    directory=train_path,
    x_col="id",
    y_col=classes,
    subset="training",
    batch_size=32,
    seed=123,
    shuffle=False,
    class_mode="raw",
    target_size=(456,456))


STEP_SIZE_TRAIN = train_generator.n // train_generator.batch_size

valid_generator = valid_datagen.flow_from_dataframe(
    dataframe=traindf,
    directory=train_path,
    x_col="id",
    y_col=classes,
    subset="validation",
    batch_size=32,
    seed=123,
    shuffle=False,
    class_mode="raw",
    target_size=(456,456))

STEP_SIZE_VALID = valid_generator.n // valid_generator.batch_size

In [ ]:
mean = np.array([123.68, 116.779, 103.939], dtype="float32")
train_generator.mean=train_generator
valid_generator.mean=valid_generator

In [ ]:
from keras.models import Sequential, Model,load_model
from keras.applications.resnet import ResNet152
from keras.applications.resnet_v2 import ResNet50V2
from keras.applications.inception_resnet_v2 import InceptionResNetV2
from keras.applications.vgg19 import VGG19
from keras.layers import Flatten, Dense, Add, Multiply, Lambda, Input, Reshape, Conv2D, DepthwiseConv2D, MaxPooling2D, BatchNormalization, Activation, ZeroPadding2D, GlobalAveragePooling2D, GlobalMaxPooling2D, Dropout, AveragePooling2D
from keras.optimizers import SGD, RMSprop, Adam, Adagrad
from keras.initializers import glorot_uniform

In [ ]:
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    strategy = tf.distribute.get_strategy()

In [ ]:
import keras.backend as backend

with strategy.scope():
  def build_model():
      vgg16 = VGG19(weights='imagenet',include_top=False,input_shape=(224,224,3))
      X = GlobalAveragePooling2D()(vgg16.output)
      X = Dense(7)(X)
      X = Activation('softmax')(X)
      model = Model(inputs = vgg16.input, outputs = X)
      opt = Adam(lr=1e-3)
      model.compile(loss='categorical_crossentropy',
                  optimizer=opt, 
                  metrics=['accuracy'])
      return model
model = build_model()
model.summary()

In [ ]:
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping, ReduceLROnPlateau, CSVLogger


class LossHistory(Callback):
    def on_train_begin(self, logs={}):
        self.losses = []
        self.val_losses = []

    def on_batch_end(self, batch, logs={}):
        self.losses.append(logs.get('loss'))
        self.val_losses.append(logs.get('val_loss'))


early_stopping = EarlyStopping(
    monitor='val_loss', patience=10, verbose=1, mode='auto')

history = LossHistory()

from keras.callbacks import ModelCheckpoint
checkpointer = ModelCheckpoint(
    filepath='/content/drive/My Drive/Classification/results/weights_vgg19.hdf5', verbose=2, save_best_only=True)

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                              patience=4,)

csv_logger = CSVLogger('/content/drive/My Drive/Classification/results/training_vgg19.csv')

In [ ]:
hist = model.fit_generator(
    train_generator,
    steps_per_epoch=STEP_SIZE_TRAIN,
    validation_data=valid_generator,
    validation_steps=STEP_SIZE_VALID,
    epochs=30,
    callbacks=[history, checkpointer, reduce_lr, early_stopping, csv_logger])

In [ ]:
model = load_model('/content/drive/My Drive/Classification/results/weights_vgg19.hdf5')

In [ ]:
test_datagen = ImageDataGenerator(
    rescale=1. / 255,
    preprocessing_function = random_input,
    )

test_generator = test_datagen.flow_from_directory(
    "/content/images/test",
    class_mode=None,
    color_mode="rgb",
    batch_size=32,
    target_size=(256, 256),
    seed=123,
    shuffle=False)

In [ ]:
test_generator.reset()

Y_pred = model.predict_generator(
    test_generator,
    steps=test_generator.n / test_generator.batch_size,
    verbose=1)

y_pred = np.argmax(Y_pred, axis=1)

In [ ]:
classification_report = classification_report(test_generator.classes, y_pred, target_names=classes)

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

cf=confusion_matrix(test_generator.classes, y_pred)

In [ ]:
cf

In [ ]:
print(classification_report)